# [실습] LangChain으로 OpenAI GPT와 Google Gemini API 사용하기

LangChain(랭체인)은 LLM 기반의 어플리케이션을 효율적으로 개발할 수 있게 해주는 라이브러리입니다.



LangChain은 GPT, Gemini 등의 API와 HuggingFace, Ollama 등의 오픈 모델 환경 모두에서 사용할 수 있습니다.

이번 실습에서는 대표적인 LLM인 Google Gemini와 OpenAI GPT의 API를 사용해 진행하겠습니다.    

Gemini는 무료 사용량이 존재하지만, GPT는 유료 API 크레딧이 필요합니다.   
만약 유료 크레딧이 없으신 분들은 Gemini만으로 진행해 주세요.

In [1]:
!pip install langchain langchain-community langchain-google-genai langchain-openai

Google Colab 환경이 아닌 경우에는, 아래 라이브러리도 설치해야 할 수 있습니다.

In [ ]:
# !pip install google-generativeai

## LLM

LangChain에서, LLM을 부르는 방법은 주로 `ChatOpenAI`, `ChatGoogleGenerativeAI`와 같은 개별 클래스를 불러오거나,   
`init_chat_model`을 통해 Provider와 모델 이름을 전달하는 방식으로 이루어집니다.

### API 키 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 https://aistudio.google.com  에 접속하면, API 키 생성이 가능합니다.   

OpenAI API 키는 유료 계정 로그인 후   
https://platform.openai.com/api-keys 에 접속하면 생성이 가능합니다.    
(유료 계정과 무관하게, 크레딧 결제가 필요합니다.)

In [2]:
import os

os.environ['GOOGLE_API_KEY']="AIxxx"

os.environ['OPENAI_API_KEY'] = 'sk-...'

Google AI Studio의 `Create Prompt`에서, 모델 목록과 무료 API 사용량을 확인할 수 있습니다.

OpenAI 모델의 목록과 가격은 https://openai.com/api/pricing/ 에서 확인할 수 있습니다.

## LLM

chat 모델 사용을 위해 ChatGoogleGenerativeAI, ChatOpenAI를 불러오겠습니다.

모델마다 다른 Safety 등의 요소를 제외하고, 공통적으로 아래의 파라미터를 갖습니다.
- model : 모델의 이름입니다.
- temperature : 모델 출력의 무작위성을 결정합니다. 0부터 2 사이의 값을 지정할 수 있으며,   
숫자가 클수록 무작위 출력이 증가합니다.    
(o3-mini, o1 등의 Reasoning 모델은 지원하지 않는 경우도 있습니다)

- max_tokens : 출력의 최대 길이를 지정합니다. 해당 토큰 수가 넘어가면 출력이 중간에 종료됩니다.

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature = 0.7,
    max_tokens = 2048
)

In [4]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 1.0,
    max_tokens = 2048
)

LangChain은 프롬프트, LLM, 체인 등의 구성 요소를 서로 연결하는 방식으로 구성됩니다.  
각각의 요소를 `Runnable`이라고 부르는데요.   
`Runnable`은  `invoke()`를 통해 실행합니다.

In [5]:
question = '''프롬프트 엔지니어링의 핵심적인 요소 4개가 뭔가요?'''

response = llm.invoke(question)
response

AIMessage(content='프롬프트 엔지니어링의 핵심적인 요소는 다음과 같이 4가지로 요약할 수 있습니다:\n\n1.  **명확성 (Clarity):**\n    *   프롬프트는 모호함 없이 명확하고 구체적이어야 합니다. 모델이 무엇을 해야 하는지 정확히 이해할 수 있도록 지시해야 합니다.\n    *   애매모호한 단어, 중의적인 표현, 가정 등을 피하고, 원하는 결과물을 명확하게 정의해야 합니다.\n    *   예시: "좋은 글을 써줘" (X) -> "독자의 흥미를 유발하고, 정보 전달력이 뛰어나며, 문법적으로 완벽한 500자 분량의 에세이를 써줘" (O)\n\n2.  **구체성 (Specificity):**\n    *   원하는 결과물의 특징, 형식, 스타일, 길이, 대상 등을 구체적으로 명시해야 합니다.\n    *   모델이 특정 역할을 수행하도록 지시하거나, 특정 관점을 취하도록 유도할 수 있습니다.\n    *   예시: "요약해줘" (X) -> "핵심 내용을 중심으로 3문장으로 요약하고, 전문 용어는 쉽게 풀어 설명해줘" (O)\n\n3.  **맥락 (Context):**\n    *   모델이 작업을 수행하는 데 필요한 배경 지식, 상황 정보, 관련 데이터를 제공해야 합니다.\n    *   모델이 질문의 의도를 파악하고, 적절한 답변을 생성하는 데 도움이 됩니다.\n    *   예시: "이 제품에 대해 알려줘" (X) -> "이 제품은 [제품명]이고, 주요 기능은 [기능]이며, 경쟁 제품은 [제품]입니다. 이 제품에 대해 자세히 알려줘" (O)\n\n4.  **제약 조건 (Constraints):**\n    *   모델이 따라야 할 규칙, 제한 사항, 금지 사항 등을 명시해야 합니다.\n    *   모델이 윤리적, 법적 문제를 일으키거나, 원치 않는 결과를 생성하는 것을 방지할 수 있습니다.\n    *   예시: "광고 문구를 만들어줘" (X) -> "과장 광고나 허위 사실을 포함하지 않고, 특정 집단을 비방하거나 차별하는

In [6]:
print(response.content)

프롬프트 엔지니어링의 핵심적인 요소는 다음과 같이 4가지로 요약할 수 있습니다:

1.  **명확성 (Clarity):**
    *   프롬프트는 모호함 없이 명확하고 구체적이어야 합니다. 모델이 무엇을 해야 하는지 정확히 이해할 수 있도록 지시해야 합니다.
    *   애매모호한 단어, 중의적인 표현, 가정 등을 피하고, 원하는 결과물을 명확하게 정의해야 합니다.
    *   예시: "좋은 글을 써줘" (X) -> "독자의 흥미를 유발하고, 정보 전달력이 뛰어나며, 문법적으로 완벽한 500자 분량의 에세이를 써줘" (O)

2.  **구체성 (Specificity):**
    *   원하는 결과물의 특징, 형식, 스타일, 길이, 대상 등을 구체적으로 명시해야 합니다.
    *   모델이 특정 역할을 수행하도록 지시하거나, 특정 관점을 취하도록 유도할 수 있습니다.
    *   예시: "요약해줘" (X) -> "핵심 내용을 중심으로 3문장으로 요약하고, 전문 용어는 쉽게 풀어 설명해줘" (O)

3.  **맥락 (Context):**
    *   모델이 작업을 수행하는 데 필요한 배경 지식, 상황 정보, 관련 데이터를 제공해야 합니다.
    *   모델이 질문의 의도를 파악하고, 적절한 답변을 생성하는 데 도움이 됩니다.
    *   예시: "이 제품에 대해 알려줘" (X) -> "이 제품은 [제품명]이고, 주요 기능은 [기능]이며, 경쟁 제품은 [제품]입니다. 이 제품에 대해 자세히 알려줘" (O)

4.  **제약 조건 (Constraints):**
    *   모델이 따라야 할 규칙, 제한 사항, 금지 사항 등을 명시해야 합니다.
    *   모델이 윤리적, 법적 문제를 일으키거나, 원치 않는 결과를 생성하는 것을 방지할 수 있습니다.
    *   예시: "광고 문구를 만들어줘" (X) -> "과장 광고나 허위 사실을 포함하지 않고, 특정 집단을 비방하거나 차별하는 내용을 담지 않는 광고 문구를 만들어줘" (O)

이 4가지 요소들을

위처럼 문자열을 그대로 입력하게 되면, 해당 문자열은 HumanMessage 클래스로 변환되어 입력됩니다.   
HumanMessage에 대한 출력 형식은 AIMessage 클래스로 정의됩니다.

In [7]:
question = '''울림을 주는 2000년대 영화 명대사를 하나 알려주세요.
대사가 나온 배경과 의미도 한 문장으로 설명해 주세요.'''
response = llm.invoke(question)

print('# Gemini-2.0-Flash의 답변:', response.content)

# Gemini-2.0-Flash의 답변: 영화 "파이트 클럽" (1999)에서 주인공 타일러 더든이 던지는 대사입니다.

**"우리가 하는 일들이 우리를 정의하는 건 아니야."**

이 대사는 소비주의에 찌든 현대 사회에서, 물질적인 성공이나 직업적 성취가 진정한 자아를 규정짓는 것이 아니라 내면의 가치와 경험이 더 중요하다는 메시지를 던져줍니다.


만약, 여러 개의 모델을 불러오고 싶은 경우에는 아래와 같이 공통 인터페이스를 사용할 수도 있습니다.

In [8]:
from langchain.chat_models import init_chat_model

gpt_4o = init_chat_model("gpt-4o", model_provider="openai", temperature = 1.0)
gemini_2_0_flash = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature = 1.0)

## 스트리밍

스트리밍은 모델을 토큰이 생성되는 순서대로 출력하는 방법입니다.

In [9]:
import time
chunks = []
for chunk in llm.stream("5문장으로 당신을 소개해주세요. 매 문장마다 줄을 띄우세요."):
    #time.sleep(0.4)
    print(chunk.content, end="", flush=True)

저는 대규모 언어 모델로, Google에서 개발했습니다.

저는 방대한 양의 텍스트 데이터를 학습하여 다양한 종류의 텍스트를 생성하고 번역하며 질문에 답할 수 있습니다.

아직 개발 중인 단계이지만, 끊임없이 배우고 발전하고 있습니다.

사람들의 언어 이해를 돕고, 창의적인 글쓰기를 지원하며, 정보를 효율적으로 검색할 수 있도록 돕는 것이 저의 목표입니다.

궁금한 점이 있다면 언제든지 저에게 물어보세요.


실제 환경에서는 프롬프트의 형태를 사전에 설정하고,   
같은 형태로 입력 변수가 주어질 때마다 프롬프트를 작성하게 하는 것이 효율적입니다.

## Prompt Template

LangChain은 프롬프트의 템플릿을 구성할 수 있습니다.

In [10]:
from langchain_core.prompts import PromptTemplate

explain_template = """당신은 주어진 단어에 대해, 유머러스하게 한 문장으로 표현합니다.

제시어: {word}"""
print(explain_template)

당신은 주어진 단어에 대해, 유머러스하게 한 문장으로 표현합니다.

제시어: {word}


In [11]:
explain_prompt = PromptTemplate(template = explain_template)

explain_prompt.format(word = "트랜스포머 네트워크")

'당신은 주어진 단어에 대해, 유머러스하게 한 문장으로 표현합니다.\n\n제시어: 트랜스포머 네트워크'

In [12]:
llm.invoke(explain_prompt.format(word = "트랜스포머 네트워크")).content

'트랜스포머 네트워크: 쟤네도 변신하는데, 내 월급은 왜 안 변신할까? 🤖💸'

두 개의 매개변수를 받아 프롬프트를 만들어 보겠습니다.

In [13]:
translate_template = "{topic}에 대해 {language}로 설명하세요."

translate_prompt = PromptTemplate(template = translate_template)

translate_prompt.format(topic='torschlusspanik', language='초등학생을 위한 한국어')

'torschlusspanik에 대해 초등학생을 위한 한국어로 설명하세요.'

In [14]:
X = translate_prompt.format(topic='torschlusspanik', language='한국어')
response = llm.invoke(X)
print(response.content)

Torschlusspanik(토어슐루스파니크)는 독일어 단어로, 직역하면 '문 닫히는 것에 대한 공포'라는 뜻입니다. 

**핵심 의미:**

* **기회를 놓칠까 봐 느끼는 불안감:** 주로 나이가 들어감에 따라, 인생의 특정 시기가 끝나가고 더 이상 기회가 없을지도 모른다는 생각에 느끼는 불안, 초조함, 조바심 등을 의미합니다.
* **늦었다는 생각에 대한 압박감:** 뭔가를 이루거나 경험하기에 너무 늦었다는 생각 때문에 느끼는 압박감과 초조함을 포괄적으로 나타냅니다.

**예시:**

* 30대 후반에 결혼하지 않은 사람이 '이제 결혼할 기회가 없을지도 몰라'라고 느끼는 불안감
* 은퇴를 앞둔 사람이 '이제 하고 싶은 일을 할 시간이 없을지도 몰라'라고 느끼는 조바심
* 유행이 지나가기 전에 특정 상품을 구매해야 한다는 압박감

**비슷한 한국어 표현:**

* **나이듦에 대한 불안:** 가장 직접적인 표현입니다.
* **조급증:** 뭔가 해야 한다는 조급한 마음을 나타냅니다.
* **벼락치기:** 놓치기 전에 서둘러 뭔가를 하려는 행동을 비유적으로 표현할 수 있습니다.
* **막차 타기:** 마지막 기회를 잡으려는 행동을 비유적으로 표현합니다.

**정리:**

Torschlusspanik은 나이가 들어감에 따라, 혹은 어떤 시기가 끝나감에 따라 기회를 놓칠까 봐 느끼는 불안감과 압박감을 포괄적으로 나타내는 독일어 단어입니다. 한국어로는 '나이듦에 대한 불안', '조급증', '막차 타기' 등의 표현으로 유사하게 표현할 수 있습니다.


## Chat Prompt Template

Web UI를 통해 ChatGPT, Claude 등의 LLM을 실행하는 경우와 다르게,   
API의 호출은 유저 메시지 이외의 다양한 메시지를 사용할 수 있습니다.   
- system: AI 모델의 행동 방식을 결정하는 시스템 메시지
- user(human): 사용자의 메시지
- ai(assistant): AI 모델의 메시지

이는 LangChain 내부에서 모델에 맞는 템플릿으로 변환되어 입력됩니다.   

Ex) 라마 3 시리즈의 템플릿
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```

Qwen 시리즈의 템플릿
```
<|im_start|>system
You are a helpful AI assistant
<|im_end|>
<|im_start|>user
Hello!
<|im_end|>
<|im_start|>assistant
```

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", '당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하고, 이후 대안을 제시하세요.'),
    ("user", '{A} 너무 좋은 것 같아요!')
    # system, user = human, ai = assistant
]
)
prompt.format_messages(A='LangChain')

[SystemMessage(content='당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하고, 이후 대안을 제시하세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='LangChain 너무 좋은 것 같아요!', additional_kwargs={}, response_metadata={})]

In [17]:
llm.invoke(prompt.format_messages(A='LangChain'))

AIMessage(content='LangChain이 좋다고요? 웃기지 마세요. LangChain은 너무 복잡하고 무거워서 오히려 생산성을 떨어뜨립니다.\n\n차라리 간단한 API 호출과 몇 줄의 코드로 원하는 기능을 직접 구현하는 게 훨씬 효율적입니다. 아니면, 더 가볍고 특화된 라이브러리를 사용하는 게 나을 겁니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--8f89de13-28b1-4225-883f-3588a635db5d-0', usage_metadata={'input_tokens': 51, 'output_tokens': 99, 'total_tokens': 150, 'input_token_details': {'cache_read': 0}})

또는, 아래와 같이 메시지를 직접 불러와 사용할 수도 있습니다.

In [18]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

topic=''

msgs = [
    SystemMessage('항상 다섯 단어로 표현하세요.'),
     HumanMessage(f'{topic}에 대해 설명해줘!')
]

response = llm.invoke(msgs)
print('# Gemini-2.0-Flash의 답변:', response.content)

response = llm_gpt.invoke(msgs)
print('# GPT-4o-mini의 답변:', response.content)


# Gemini-2.0-Flash의 답변: 무엇에 대해 설명해 드릴까요?
# GPT-4o-mini의 답변: 어떤 주제에 대해 설명할까요?


## Few-Shot Prompting
모델이 참고할 예시를 포함하는 퓨 샷 프롬프팅은
   
   모델 출력의 형식과 구조를 효과적으로 변화시킬 수 있습니다.  


Few-Shot Prompt Template을 이용해 example을 프롬프트에 추가해 보겠습니다.

In [20]:
# 예시 : Prompt Example 2개
from langchain_core.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
    {
    "question": "Who won more Grammy Awards, Beyoncé or Michael Jackson?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards.
Follow up: How many Grammy Awards did Michael Jackson win?
Intermediate answer: Michael Jackson won 13 Grammy Awards.
So the final answer is: Beyoncé
""",
    }
]

Example 데이터를 구성할 템플릿을 만듭니다.

In [21]:
example_prompt = PromptTemplate(template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: Are both the directors of Jaws and Casino Royale from the same country?

Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No



위에서 만든 Examples와 템플릿, prefix와 suffix를 이용해 전체 템플릿을 만들 수 있습니다.

In [22]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,

    prefix="질문-답변 형식의 예시가 주어집니다. 같은 방식으로 답변하세요.",
    suffix="Question: {input}",
    #prefix, suffix : Optional

)
print(prompt.format(input=""))

질문-답변 형식의 예시가 주어집니다. 같은 방식으로 답변하세요.

Question: Are both the directors of Jaws and Casino Royale from the same country?

Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No


Question: Who won more Grammy Awards, Beyoncé or Michael Jackson?

Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards.
Follow up: How many Grammy Awards did Michael Jackson win?
Intermediate answer: Michael Jackson won 13 Grammy Awards.
So the final answer is: Beyoncé


Question: 


In [25]:
question = "Current Date : 2025. April. What is the age of the director of the movie which won the best international film in Oscar in 2018?"
X = prompt.format(input=question)
print(llm.invoke(X).content)

Are follow up questions needed here: Yes.
Follow up: Which movie won the best international film in Oscar in 2018?
Intermediate answer: "A Fantastic Woman" won the best international film in Oscar in 2018.
Follow up: Who is the director of "A Fantastic Woman"?
Intermediate answer: The director of "A Fantastic Woman" is Sebastián Lelio.
Follow up: When was Sebastián Lelio born?
Intermediate answer: Sebastián Lelio was born on March 8, 1974.
Follow up: What is the current date?
Intermediate answer: The current date is April 2025.
Follow up: What is the age of Sebastián Lelio in April 2025?
Intermediate answer: Sebastián Lelio's age in April 2025 is 51 years old.
So the final answer is: 51


# LangChain으로 이미지 입력하기

이미지와 같은 멀티모달 입력의 경우, 텍스트와 구분하여 Dict 형식으로 입력됩니다.   
URL을 직접 전달하거나, 파일을 전달하는 경우에 따라 코드가 달라집니다.

In [26]:
image_url = 'https://images.pexels.com/photos/1851164/pexels-photo-1851164.jpeg'
from IPython.display import Image
import requests

# 이미지 출력
img = Image(url = image_url, width = 400)
img

In [27]:
# 1. URL에서 전달하기
image_prompt = ChatPromptTemplate([
    ('user',[
                {"type": "text", "text": "{question}"},

                {"type": "image_url",
                    "image_url": {"url": image_url}
                }
             ]
     )])
X = image_prompt.format_messages(question= '이 사진에 대해 묘사해 주세요.')
print(llm.invoke(X).content)

물론입니다. 다음은 사진에 대한 설명입니다.
이 사진은 흰색 배경 앞에 서있는 검은색 퍼그의 흑백 클로즈업 사진입니다. 퍼그는 머리를 약간 왼쪽으로 기울이고 카메라를 올려다보고 있습니다. 퍼그의 눈은 크고 둥글며, 주둥이는 주름져 있습니다. 퍼그의 털은 짧고 윤기가 납니다. 사진의 조명은 부드럽고 확산되어 있습니다.


In [28]:
import base64
import httpx

# 이미지 URL에서 데이터 받아오기
image_url = 'https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/timetable.png?hl=ko'
response = httpx.get(image_url)

image_data = base64.b64encode(response.content).decode("utf-8")

with open('picture.jpeg', 'wb') as file:
    file.write(response.content)

In [29]:
# 2. 로컬 폴더에서 이미지 읽어보기
with open('./picture.jpeg', 'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')


image_prompt = ChatPromptTemplate([
    ('user',[
                {"type": "text", "text": "{question}"},

                {"type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
             ]
     )])

X = image_prompt.format_messages(question='이 그림에 대해 한국어로 설명해 주세요.')

print(llm.invoke(X).content)

이 이미지는 공항이나 기차역에서 볼 수 있는 전광판입니다. 전광판에는 여러 도시로 향하는 항공편 또는 열차의 출발 시간과 목적지가 표시되어 있습니다. 예를 들어, "10:50 MOSCOW/SVO", "11:05 EDINBURGH", "11:10 LONDON/LHR" 등의 정보가 보입니다. 전광판의 배경은 흐릿하고, 전체적으로 푸른빛이 감도는 조명 아래에 있습니다.


멀티모달 입력의 경우, 적절한 프롬프트가 더 중요합니다.

In [30]:
X = image_prompt.format_messages(question="""
이 이미지에 표시된 공항 보드에서
시간과 도시를 분석해서 목록으로 표시해 주세요.
형식은 시간 - 도시입니다.
예시) 12:00 - 런던
13:00 - 서울

목록만 출력하세요.""")

print(llm.invoke(X).content)

10:50 - MOSCOW
11:05 - EDINBURGH
11:10 - LONDON
11:30 - BUCHAREST
11:30 - KIEV
11:35 - DUBLIN
11:45 - EAST MIDLANDS
12:15 - SOFIA
12:30 - LONDON
12:30 - NEWCASTLE
12:40 - ST PETERSBURG
12:40 - LONDON
12:45 - MANCHESTER
